# 1. Introdução

o ecossistema Hadoop é um conjunto de tecnologias de código aberto projetado para armazenar, processar, analisar e gerenciar grandes volumes de dados de forma distribuída. Ele é amplamente utilizado para lidar com os desafios relacionados ao processamento de dados em escala, como os gerados por empresas, instituições de pesquisa e diversas indústrias.
O componente central do ecossistema Hadoop é o Hadoop Distributed File System (HDFS), um sistema de arquivos distribuído que divide os dados em blocos e os distribui em vários nós de um cluster de servidores. Isso permite o armazenamento e processamento eficiente de grandes conjuntos de dados.

Por sua vez, o Apache Spark é um framework de computação distribuída de código aberto projetado para processamento de dados em grande escala. Ele fornece uma plataforma unificada para processamento de dados em lote, processamento em tempo real, aprendizado de máquina e análise interativa. O Spark foi projetado para ser rápido, flexível e fácil de usar.

Neste projeto, buscou-se investigar a aplicação prática do Apache Spark para a análise das informações fornecidas pelos dados de músicas e artistas do Spotify, buscando compreender como esse ecossistema pode ser empregado na extração de insights valiosos a partir de grandes volumes de dados. Ao explorar as funcionalidades do Spark e sua integração com os dados da plataforma de streaming, esperou-se obter uma visão mais profunda das capacidades analíticas que essa combinação pode oferecer.


# 2. Transformação dos dados

A seguir, iremos inicializar o Apache Spark e ler os arquivos *artists.csv* e *tracks.csv*. Depois, iremos fazer uma pequena transformação nos dados no dataframe de tracks, fazendo a conversão de tipagem de algumas colunas e, por fim, unindo este ao dataframe de artists.


In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import (avg, col, count, date_format, explode, rank,
                                   regexp_replace, split, sum)

spark = SparkSession.builder.getOrCreate()


your 131072x1 screen size is bogus. expect trouble
23/10/11 19:04:10 WARN Utils: Your hostname, geazi resolves to a loopback address: 127.0.1.1; using 192.168.75.156 instead (on interface eth0)
23/10/11 19:04:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/11 19:04:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
raw_artists = spark.read.csv(
    "artists.csv",
    header=True,
    inferSchema=True
)

raw_artists.printSchema()


root
 |-- id: string (nullable = true)
 |-- followers: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)



In [3]:
raw_tracks = spark.read.csv(
    "tracks.csv",
    header=True,
    inferSchema=True
)

raw_tracks.printSchema()


root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- id_artists: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)



In [4]:
transformed_tracks = raw_tracks.select(
    "id",
    "name",
    col("popularity").cast("INTEGER").alias("popularity"),
    col("duration_ms").cast("INTEGER").alias("duration_ms"),
    col("release_date").cast("DATE").alias("release_date"),
    explode(split(regexp_replace("id_artists", "[\\[\\]']", ""), ",")).alias(
        "id_artist")
)

transformed_tracks.printSchema()


root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- release_date: date (nullable = true)
 |-- id_artist: string (nullable = false)



In [5]:
raw_artists = raw_artists.alias("a")
transformed_tracks = transformed_tracks.alias("t")

tracks = (
    transformed_tracks.join(raw_artists, on=transformed_tracks.id_artist == raw_artists.id)
    .select(
        col("a.id").alias("artist_id"),
        col("a.name").alias("artist_name"),
        col("a.popularity").alias("artist_poppularity"),
        col("a.followers").alias("artist_followers"),
        col("t.id").alias("track_id"),
        col("t.name").alias("track_name"),
        col("t.popularity").alias("track_popularity"),
        col("t.duration_ms").alias("track_duration_ms"),
        col("t.release_date").alias("track_release_date")
    )
)

tracks.printSchema()


root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_poppularity: string (nullable = true)
 |-- artist_followers: double (nullable = true)
 |-- track_id: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_popularity: integer (nullable = true)
 |-- track_duration_ms: integer (nullable = true)
 |-- track_release_date: date (nullable = true)



# 3. Análise dos dados

Nas próximas células de código iremos fazer algumas análises de dados com base nas análises feitas anteriormente com o Apache Hive. Para cada análise será explanado o problema seguido imediatamente pelo trecho de código.


## 3.1. Artistas com músicas mais populares

Com essa análise, buscou-se analisar os três principais artistas que tiveram a maior somatória de popularidade de suas músicas e que ano em que esse record ocorreu.


In [7]:
ex1 = (
    tracks.withColumn("year", date_format("track_release_date", "y"))
    .groupBy("artist_id", "artist_name", "year")
    .agg(sum("track_popularity").alias("total_popularity"))
    .orderBy("total_popularity", ascending=False)
    .limit(3)
)

ex1.show(truncate=False)


+----------------------+-----------------+----+----------------+
|artist_id             |artist_name      |year|total_popularity|
+----------------------+-----------------+----+----------------+
|3meJIgRw7YleJrmbpbJK6S|Die drei ???     |1980|16710           |
|3meJIgRw7YleJrmbpbJK6S|Die drei ???     |1979|15918           |
|0i38tQX5j4gZ0KS3eCMoIl|TKKG Retro-Archiv|1982|13030           |
+----------------------+-----------------+----+----------------+



## 3.2. Média da duração das músicas

Essa análise teve como objetivo analisar os dez principais artistas cuja somatória da duração de suas músicas fosse no máximo 50% maior do que a média geral da duração das músicas de todos os artistas.


In [8]:
avg_duration_ms = tracks.select(avg("track_duration_ms").cast(
    "INTEGER").alias("track_avg_duration_ms")).first().track_avg_duration_ms

ex2 = (
    tracks.filter(tracks.track_duration_ms >=
                  avg_duration_ms + avg_duration_ms * 0.5)
    .select("track_name", (col("track_duration_ms") / 60).alias("track_duration_s"))
    .orderBy("track_duration_s", ascending=False)
    .limit(10)
)

ex2.show(truncate=False)


+------------------------------------------------------------+-----------------+
|track_name                                                  |track_duration_s |
+------------------------------------------------------------+-----------------+
|โครงสร้างแห่งสิ่งที่เรียกว่าชีวิต                           |93686.96666666666|
|Surah Al-Araf                                               |83251.38333333333|
|Tech House The Yearbook 2018 - Continuous Mix 2             |81072.21666666666|
|6 Years of 1980 Recordings - Part 2 - 2010-2012 Dj Mix      |80001.96666666666|
|Surah An-Nisa                                               |79954.3          |
|Monster Tunes Yearmix 2011 - Mixed by Mark Eteson           |79876.45         |
|Tech House The Yearbook 2018 - Continuous Mix 1             |79277.03333333334|
|Surah Aal-E-Imran                                           |78754.4          |
|Happy New Year Mix 2009                                     |78278.16666666667|
|Enhanced Best of 2010 Year 

## 3.3. Artista com mais seguidores

Dessa vez, analisou-se o artista com a maior quantidade de seguidores e quais de suas músicas é a mais popular.


In [9]:
window_spec = Window.partitionBy("artist_id").orderBy(
    col("track_popularity").desc())

ex3 = (
    tracks.withColumn("rank", rank().over(window_spec))
    .filter("rank == 1")
    .orderBy("artist_followers", ascending=False)
    .select("artist_name", "artist_followers", col("track_name").alias("most_popular_track"))
    .limit(5)
)

ex3.show(5, truncate=False)


+-------------+----------------+--------------------------------------+
|artist_name  |artist_followers|most_popular_track                    |
+-------------+----------------+--------------------------------------+
|Ed Sheeran   |7.8900234E7     |Afterglow                             |
|Ariana Grande|6.1301006E7     |positions                             |
|Drake        |5.4416812E7     |What’s Next                           |
|Justin Bieber|4.4606973E7     |Peaches (feat. Daniel Caesar & Giveon)|
|Eminem       |4.3747833E7     |Godzilla (feat. Juice WRLD)           |
+-------------+----------------+--------------------------------------+



## 3.4. Décadas com mais lançamento de músicas

Com essa análise, procurou determinar as décadas que tiveram a maior quantidade de lançamento de músicas, assim como as décadas que tiveram mais músicas populares.


In [10]:
ex4 = (
    tracks.withColumn("track_release_year", date_format(
        "track_release_date", "y").cast("INTEGER"))
    .withColumn("decade", col("track_release_year") - col("track_release_year") % 10)
    .groupBy("decade")
    .agg(
        count("*").alias("total_released_tracks"),
        sum("track_popularity").alias("total_popularity")
    )
    .orderBy("total_popularity", ascending=False)
)

ex4.show(truncate=False)


+------+---------------------+----------------+
|decade|total_released_tracks|total_popularity|
+------+---------------------+----------------+
|2010  |100860               |4058934         |
|1990  |107203               |3172233         |
|2000  |85399                |3145675         |
|1980  |81390                |2097852         |
|1970  |60984                |1481596         |
|1960  |46339                |837660          |
|2020  |17782                |794987          |
|1950  |33349                |290986          |
|1940  |17722                |31960           |
|1930  |12922                |27469           |
|1920  |7583                 |8660            |
|1900  |1                    |19              |
+------+---------------------+----------------+



## 3.5. Música mais popular da década

Por fim, essa análise identifica qual foi a música mais popular da década, desde 1921 até 2020.


In [11]:
window_spec = Window.partitionBy("decade").orderBy(
    col("track_popularity").desc(), col("track_id"))

ex5 = (
    tracks.withColumn("track_release_year", date_format(
        "track_release_date", "y").cast("INTEGER"))
    .withColumn("decade", col("track_release_year") - col("track_release_year") % 10)
    .withColumn("rank", rank().over(window_spec))
    .filter("rank == 1")
    .select("decade", col("track_name").alias("most_popular_track"), "track_popularity")
    .distinct()
    .orderBy("decade", ascending=False)
)

ex5.show(truncate=False)


+------+----------------------------------------+----------------+
|decade|most_popular_track                      |track_popularity|
+------+----------------------------------------+----------------+
|2020  |Peaches (feat. Daniel Caesar & Giveon)  |100             |
|2010  |Streets                                 |94              |
|2000  |Cupid's Chokehold / Breakfast in America|87              |
|1990  |Smells Like Teen Spirit                 |83              |
|1980  |Take on Me                              |86              |
|1970  |Dreams - 2004 Remaster                  |86              |
|1960  |Fortunate Son                           |83              |
|1950  |Johnny B. Goode                         |77              |
|1940  |Gymnopédie No. 1                        |69              |
|1930  |All of Me                               |65              |
|1920  |Mack the Knife                          |55              |
|1900  |Maldita sea la primera vez              |19           

# 4. Considerações finais e próximos passos

A análise de dados realizada utilizando o Apache Spark em um conjunto de dados provenientes do Spotify demonstrou a capacidade poderosa dessa ferramenta no processamento e exploração de informações em larga escala. A combinação das tabelas "tracks" e "artists" proporcionou uma visão abrangente sobre as tendências musicais, popularidade das faixas e artistas, bem como padrões de lançamento e duração de músicas.

Através da execução de consultas complexas e janelas de análise no Spark, foi possível identificar insights valiosos que podem orientar decisões estratégicas no cenário da indústria musical. Por exemplo, ao analisar a popularidade das faixas por década, pudemos observar como os gostos musicais evoluíram ao longo do tempo, destacando os artistas e gêneros mais influentes em cada período.

Uma das aplicações promissoras dos dados coletados, como uma proposta de solução futura, é a construção de um modelo de machine learning para prever a popularidade futura de novas faixas com base nas características dos artistas, gêneros e outras variáveis relevantes. Utilizando algoritmos de regressão ou classificação, é possível criar um modelo que ajude as gravadoras e plataformas de streaming a direcionar seus esforços de promoção e recomendação de músicas, maximizando o impacto e alcance das novas faixas.
